# PROJECT: EXTRACTION OF NETFLIX DATA 


In [2]:
!pip install sqlalchemy


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install pandas pyodbc



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = df.where(pd.notnull(df), None)


In [1]:
import pandas as pd
import pyodbc

# Load data
df = pd.read_csv("netflix_raw.csv")

# Replace NaN with None
df = df.where(pd.notnull(df), None)

# SQL connection
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=NOUSHAD-ALAM;"
    "DATABASE=NetflixDB;"
    "Trusted_Connection=yes;"
)

cursor = conn.cursor()
table_name = "netflix_raw_data"

# Drop table
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
conn.commit()

# Create table
columns = [f"[{col}] NVARCHAR(MAX)" for col in df.columns]

create_table_query = f"""
CREATE TABLE {table_name} (
    {', '.join(columns)}
)
"""
cursor.execute(create_table_query)
conn.commit()

# Insert data
column_names = ', '.join([f"[{col}]" for col in df.columns])
placeholders = ', '.join(['?' for _ in df.columns])

insert_query = f"""
INSERT INTO {table_name} ({column_names})
VALUES ({placeholders})
"""

cursor.fast_executemany = True
cursor.executemany(insert_query, df.values.tolist())
conn.commit()

print("✅ Netflix raw data uploaded successfully!")

cursor.close()
conn.close()


✅ Netflix raw data uploaded successfully!
